In [20]:
import json
import pandas
import os
import re
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
from  itertools import tee, chain, islice
from math import log
from operator import itemgetter
from itertools import takewhile

In [21]:
root_dir = '/home/marcin/Desktop/SemestrVIII/PJN'
year = "2018"
json_data_dir = f"{root_dir}/data/json"
filesInYearPattern = 'judgments-(316[3-9]|317\d)\.json'

In [22]:
def window(it, size=2):
    yield from zip(*[islice(it, s, None) for s, it in enumerate(tee(it, size))])

In [23]:
def unigrams(line):
    notags = re.sub(r"<[^>]*>", " ", line)
    nobreaks =  re.sub(r"-\n", " ", notags)
    nodigits =  re.sub(r"\d+", " ", nobreaks)
    noroman =  re.sub(r"\b[XVILMC]+\b", "", nodigits)
    lower =   noroman.lower()
    return  re.findall(r'\w+', lower)

In [24]:
def make_bigrams(unigrams):
    bigram_tuples = window(unigrams,2)
    bigrams = map(lambda tuple: f"{tuple[0]} {tuple[1]}", bigram_tuples) 
    return bigrams

In [25]:
def judgement_texts(filename):
    with open(os.path.join(json_data_dir, filename), 'r') as jsonFile:
        judgements = json.load(jsonFile)['items'] 
    year_filtered = filter(lambda item: year in item['judgmentDate'], judgements)
    yield from map(lambda item: item['textContent'], year_filtered)

In [26]:
def make_bigrams(unigrams):
    bigram_tuples = window(unigrams,2)
    bigrams = map(lambda tuple: f"{tuple[0]} {tuple[1]}", bigram_tuples) 
    return bigrams

In [27]:
json_files = os.listdir(json_data_dir)
judgements_files= filter(lambda name: re.match(filesInYearPattern, name), json_files)
texts =  chain.from_iterable(map( judgement_texts, judgements_files))
unigrams_it1, unigrams_it2 =tee(map(unigrams,texts))
unigrams =  chain.from_iterable(unigrams_it1)
bigrams = chain.from_iterable(map( lambda it: window(it,2),unigrams_it2))
   

In [28]:
counted_unigrams= Counter(unigrams)
counted_bigrams= Counter(bigrams)

unigram_count = sum(counted_unigrams.values())
bigram_count =  sum(counted_bigrams.values())

# 10 najpopularniejszych bigramów w 2018

In [29]:
counted_bigrams.most_common(10)

[(('z', 'dnia'), 11869),
 (('art', 'k'), 7184),
 (('w', 'dniu'), 5208),
 (('na', 'podstawie'), 4699),
 (('k', 'p'), 4686),
 (('w', 'sprawie'), 4657),
 (('z', 'art'), 4453),
 (('w', 'tym'), 4196),
 (('art', 'ust'), 3575),
 (('zgodnie', 'z'), 3378)]

# 10 najpopularniejszych unigramów w 2018

In [30]:
counted_unigrams.most_common(10) 

[('w', 118896),
 ('z', 69578),
 ('na', 42894),
 ('i', 39428),
 ('do', 37881),
 ('nie', 30528),
 ('o', 29826),
 ('k', 28916),
 ('r', 27356),
 ('że', 22479)]

In [31]:
def pmi(bigram):
    prob_word_1 = counted_unigrams[bigram[0]] / unigram_count
    prob_word_2 = counted_unigrams[bigram[1]] / unigram_count
    prob_bigram = counted_bigrams[bigram] /     bigram_count 
    return log( prob_bigram / (prob_word_1 * prob_word_2))    

In [32]:
def top_pmi(min_frequency = 1):
    frequent_bigrams = takewhile(lambda x: x[1] > min_frequency, counted_bigrams.most_common() )
    pmi_vals = map(lambda x: (x[0],pmi(x[0])),frequent_bigrams)
    return list(sorted(pmi_vals,key = itemgetter(1),reverse=True))

# top 30 bigrams by PMI

In [33]:
top_pmi(100)[:30]

[(('samorządowej', 'jednostce'), 9.594232074185577),
 (('trybunału', 'konstytucyjnego'), 9.589584116568494),
 (('zdrowotnej', 'finansowanych'), 9.518455006864993),
 (('księgę', 'wieczystą'), 9.517003377571543),
 (('sfery', 'budżetowej'), 9.410511276635498),
 (('doznaną', 'krzywdę'), 9.405390343875302),
 (('uzasadnieniem', 'doręczyć'), 9.36187456907651),
 (('stawek', 'dziennych'), 9.240075327075525),
 (('grach', 'hazardowych'), 9.22696819871096),
 (('gospodarstwa', 'rolnego'), 9.194750897930989),
 (('jednostce', 'sfery'), 9.122246283735851),
 (('obszaru', 'ograniczonego'), 9.098896619410258),
 (('gospodarstwie', 'rolnym'), 9.026640589888276),
 (('ograniczonego', 'użytkowania'), 8.980475404382185),
 (('służby', 'wojskowej'), 8.964687070754378),
 (('szczecin', 'centrum'), 8.927170559858226),
 (('godzinach', 'nadliczbowych'), 8.923842599384564),
 (('tekst', 'jednolity'), 8.918322125917946),
 (('utraciła', 'zdolność'), 8.858359840264844),
 (('doświadczenia', 'życiowego'), 8.836677207577926)

In [34]:
top_pmi(1)[:30]

[(('ogórków', 'rozpaletyzowanych'), 13.985007455438776),
 (('delikatesów', 'mięsnych'), 13.985007455438776),
 (('czerwona', 'kropka'), 13.985007455438776),
 (('podłogach', 'grzyb'), 13.985007455438776),
 (('poplamione', 'pozaciągane'), 13.985007455438776),
 (('pozaciągane', 'wymięte'), 13.985007455438776),
 (('odczyn', 'opłucnowy'), 13.985007455438776),
 (('zaspokajających', 'najpotrzebniejsze'), 13.985007455438776),
 (('kantak', 'rekowska'), 13.985007455438776),
 (('zasypowych', 'zadaszeń'), 13.985007455438776),
 (('przenośnikiem', 'taśmowym'), 13.985007455438776),
 (('geologiczno', 'geotechnicznymi'), 13.985007455438776),
 (('ulotkami', 'reklamowymi'), 13.985007455438776),
 (('catering', 'gesellschaftmbh'), 13.985007455438776),
 (('jakiemuś', 'sezonowi'), 13.985007455438776),
 (('taneczno', 'rytmicznych'), 13.985007455438776),
 (('oferujące', 'trampoliny'), 13.985007455438776),
 (('bon', 'aktywacyjny'), 13.985007455438776),
 (('odblokowania', 'immobilizera'), 13.985007455438776),
 ((

In [35]:
from  python_llr.llr import llr_2x2
def loglikelihood(bigram):
   k11 = counted_bigrams[bigram] 
   k12 = counted_unigrams[bigram[0]] - counted_bigrams[bigram]
   k21 = counted_unigrams[bigram[1]] - counted_bigrams[bigram]
   k22 = bigram_count - (counted_unigrams[bigram[0]] + counted_unigrams[bigram[1]] - counted_bigrams[bigram])
   return llr_2x2(k11,k12,k21,k22) 

# top 30 bigrams by loglikelihood

In [36]:
log_like_vals = map(lambda bigram: (bigram, loglikelihood(bigram)) , counted_bigrams.keys())
list(sorted(log_like_vals,key = itemgetter(1),reverse=True))[:30]

[(('z', 'dnia'), 62312.695006053895),
 (('art', 'k'), 40665.28016950283),
 (('ubezpieczeń', 'społecznych'), 39026.606494446016),
 (('na', 'podstawie'), 36236.23216680449),
 (('sygn', 'akt'), 29820.272836939796),
 (('art', 'ust'), 29020.283317264926),
 (('w', 'dniu'), 28772.78955286031),
 (('k', 'p'), 28559.349020969472),
 (('p', 'c'), 27353.928227595374),
 (('sąd', 'okręgowy'), 25354.20566470563),
 (('sąd', 'rejonowy'), 22336.20282959295),
 (('zgodnie', 'z'), 21642.9404687098),
 (('na', 'rzecz'), 20272.78232333425),
 (('w', 'sprawie'), 19978.846680278424),
 (('organ', 'rentowy'), 19408.80432157178),
 (('dz', 'u'), 19348.702547280263),
 (('sądu', 'najwyższego'), 18693.15304123392),
 (('stycznia', 'r'), 16418.327593825758),
 (('o', 'pracę'), 15980.856582631415),
 (('z', 'tytułu'), 14428.569640153088),
 (('w', 'tym'), 14280.435032461071),
 (('zw', 'z'), 14121.039062798372),
 (('podstawie', 'art'), 13872.707533075416),
 (('sądu', 'rejonowego'), 13678.652261063151),
 (('ocenie', 'sądu'), 13

In [37]:
log_like_vals_weighted = map(lambda bigram: (bigram, log(counted_bigrams[bigram]) * loglikelihood(bigram)) , counted_bigrams.keys())
list(sorted(log_like_vals_weighted,key = itemgetter(1),reverse=True))

[(('z', 'dnia'), 584598.0908818131),
 (('art', 'k'), 361091.89391628955),
 (('ubezpieczeń', 'społecznych'), 307205.45205577626),
 (('na', 'podstawie'), 306381.1477421989),
 (('w', 'dniu'), 246236.12841767285),
 (('k', 'p'), 241393.17442797867),
 (('art', 'ust'), 237435.84563047942),
 (('sygn', 'akt'), 233123.72364885383),
 (('p', 'c'), 220052.12987458333),
 (('sąd', 'okręgowy'), 197870.59463703522),
 (('zgodnie', 'z'), 175849.7374902697),
 (('sąd', 'rejonowy'), 171628.344378011),
 (('w', 'sprawie'), 168743.87124024428),
 (('na', 'rzecz'), 161278.6011171186),
 (('dz', 'u'), 141960.20995148917),
 (('sądu', 'najwyższego'), 137843.34528317602),
 (('organ', 'rentowy'), 137706.71614772046),
 (('stycznia', 'r'), 127650.12897185983),
 (('o', 'pracę'), 124073.31357666483),
 (('w', 'tym'), 119125.77491630948),
 (('z', 'tytułu'), 113044.78582329376),
 (('zw', 'z'), 107667.54254251099),
 (('podstawie', 'art'), 106102.11616373528),
 (('od', 'dnia'), 102753.67497707538),
 (('k', 'c'), 102081.8789660